<a href="https://colab.research.google.com/github/Hemapriya-RC/Hema-ranjith/blob/main/Global_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

#pd.options.mode.copy_on_write = True

In [6]:
df = pd.read_csv("/content/GlobalWeatherRepository.csv")
pd.set_option('display.max_columns', None)

df.sample(5)

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,wind_mph,wind_kph,wind_degree,wind_direction,pressure_mb,pressure_in,precip_mm,precip_in,humidity,cloud,feels_like_celsius,feels_like_fahrenheit,visibility_km,visibility_miles,uv_index,gust_mph,gust_kph,air_quality_Carbon_Monoxide,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
3975,Iraq,Baghdad,33.34,44.39,Asia/Baghdad,1717510500,2024-06-04 17:15,44.0,111.2,Sunny,8.1,13.0,310,NW,1002.0,29.59,0.00,0.0,10,0,46.2,115.2,10.0,6.0,10.0,21.9,35.2,220.3,183.1,5.0,8.7,6.8,11.4,1,1,04:53 AM,07:09 PM,02:54 AM,05:07 PM,Waning Crescent,8
1025,Eritrea,Asmara,15.33,38.93,Africa/Asmara,1716216300,2024-05-20 17:45,17.3,63.1,Patchy rain nearby,4.9,7.9,136,SE,1013.0,29.93,0.05,0.0,68,84,17.3,63.1,10.0,6.0,4.0,10.3,16.6,188.6,57.2,0.3,0.7,18.0,70.6,2,2,05:54 AM,06:48 PM,04:14 PM,03:32 AM,Waxing Gibbous,88
1053,Israel,Jerusalem,31.78,35.23,Asia/Jerusalem,1716216300,2024-05-20 17:45,32.0,89.6,Sunny,12.5,20.2,290,WNW,1008.0,29.77,0.00,0.0,24,0,29.9,85.9,10.0,6.0,7.0,29.3,47.1,195.3,94.4,3.0,1.9,29.3,151.1,2,3,05:39 AM,07:33 PM,04:45 PM,03:36 AM,Waxing Gibbous,88
1352,United States of America,Washington Park,46.60,-120.49,America/Los_Angeles,1716302700,2024-05-21 07:45,11.1,52.0,Partly cloudy,15.0,24.1,310,NW,1017.0,30.04,0.00,0.0,54,50,11.0,51.8,16.0,9.0,4.0,19.5,31.3,203.6,50.1,3.5,0.2,1.3,2.5,1,1,05:21 AM,08:37 PM,07:08 PM,04:08 AM,Waxing Gibbous,93
4758,Japan,Tokyo,35.69,139.69,Asia/Tokyo,1717856100,2024-06-08 23:15,22.2,72.0,Partly cloudy,10.5,16.9,170,S,1016.0,30.00,0.00,0.0,78,25,24.6,76.4,10.0,6.0,1.0,17.1,27.5,290.4,9.9,69.2,20.7,6.5,13.3,1,1,04:25 AM,06:56 PM,05:24 AM,08:59 PM,Waxing Crescent,3


In [7]:
temperature_max = df['temperature_celsius'].max()
temperature_min = df['temperature_celsius'].min()
temperature_mean = df['temperature_celsius'].mean()

df['last_updated'] = pd.to_datetime(df['last_updated'])

last_updated = df['last_updated'].max().strftime('%Y-%m-%d %H:%M:%S')

average_temp_by_country_year = df.groupby([df['last_updated'].dt.year, 'country'])['temperature_celsius'].mean().reset_index()
average_temp_by_country_year.columns = ['year', 'country', 'average_temperature']

print(average_temp_by_country_year)

     year      country  average_temperature
0    2024  Afghanistan            21.414286
1    2024      Albania            27.425000
2    2024      Algeria            26.671429
3    2024      Andorra            13.696429
4    2024       Angola            28.146429
..    ...          ...                  ...
180  2024    Venezuela            28.317857
181  2024      Vietnam            28.364286
182  2024        Yemen            26.725000
183  2024       Zambia            24.785714
184  2024     Zimbabwe            23.653571

[185 rows x 3 columns]


In [8]:
fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value= temperature_mean,
    title={'text' : 'Temperature (°C)'},
    gauge= { 'axis' : {'range' : [None, 50]},
            'bar': {'color': "#008000"},
            'steps' : [
                {'range' : [0, 10], 'color' : "#440154"},
                {'range' : [10, 20], 'color' : "#3b528b"},
                {'range' : [20, 30], 'color' : "#21918c"},
                {'range' : [30, 40], 'color' : "#5ec962"},
                {'range' : [40, 50], 'color' : "#fde725"}
            ],
            'threshold' : {
                'line' : {'color' : 'red', 'width' : 5},
                'thickness' : 0.75,
                'value' : temperature_mean,
                            }
    }
))

fig.update_layout(
    title = "Average global temperature <br>(updated " + last_updated + ")"
)

fig.show()

In [9]:
histogram = px.histogram(
    df,
    x='temperature_celsius',
    nbins=20,
    title='Global temperature distribution',
    labels={'temperature_celsius': 'Temperature (°C)'}
)


histogram.show()

In [10]:
top_countries = average_temp_by_country_year.groupby('country')['average_temperature'].mean().nlargest(5).reset_index()
top_countries = top_countries.sort_values(by='average_temperature')

fig = px.bar(top_countries, x='average_temperature', y='country', orientation='h',
             title='Top 5 hottest countries', labels={'average_temperature': 'Average temperature (°C)', 'country': 'Countries'},
             text=top_countries['average_temperature'].round(2).astype(str) + ' °C'

)

fig.update_traces(hovertemplate='')
fig.update_traces(hoverinfo='none')

fig.show()


In [11]:
top_countries = average_temp_by_country_year.groupby('country')['average_temperature'].mean().nsmallest(5).reset_index()
top_countries = top_countries.sort_values(by='average_temperature')

fig = px.bar(top_countries, x='average_temperature', y='country', orientation='h',
             title='Top 5 coldest countries', labels={'average_temperature': 'Average temperature (°C)', 'country': 'Countries'},
             text=top_countries['average_temperature'].round(2).astype(str) + ' °C'

)

fig.update_traces(hovertemplate='')
fig.update_traces(hoverinfo='none')

fig.show()

In [12]:
fig = px.bar(
    average_temp_by_country_year,
    x='country',
    y='average_temperature',
    title='Global average temperature by country',
    labels={'temperature_celsius':'(°C)', 'country': 'Countries'},
    color='average_temperature',
    color_continuous_scale=px.colors.sequential.Viridis,
    height=800
)

fig.show()